# Análise e Histórico de Experimentos de Classificação de Categorias

Este *notebook* tem como objetivo permitir o registro visual dos experimentos feitos e oferecer um modo de recuperar um experimento e analisar os resultados em mais detalhes.
Os princípios usados para recuperar o modelo e as funções de pré-processamento usadas aqui também podem ser usadas para "produtizar" o classificador e o recomendador -- com as ressalvas já feitas anteriormente, sobre modos melhores de implementar o pipeline para o ambiente de produção e para os modos de processamento unitário ou em lote.

## Bibliotecas e Funções

In [1]:
# General
import sys
import funcy as fp
from pathlib import Path

# Visualization / Presentation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

import mlflow
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix

# Carregar, além de atualizar frequentemente, código personalizado disponível em ../src
%load_ext autoreload 
%autoreload 2
sys.path.append(str(Path.cwd().parent))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.utils.experiments import (set_dataset_split, 
                                   compute_multiclass_classification_metrics)


# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

## Recuperação do melhor resultado

A seguir, recupera-se resultado dos experimentos considerando o valor os maiores valores de F1 médio entre as categorias.

In [2]:
EXPERIMENT_ID = '0'

mlflow_client = MlflowClient()

best_experiments_result = [
    mlflow.search_runs(experiment_ids=[experiment_id], 
                       max_results=200,
                       order_by=['metrics.F1 DESC'], 
                       filter_string='attributes.status="FINISHED"')
    for experiment_id in [EXPERIMENT_ID]
]

best_results = pd.concat(best_experiments_result, axis=0)

Dada uma execução, tem-se a lista de resultados individuais dos modelos e a execução agregadora, sem nome de modelo (*params.model_name*), que mantém as informações do experimento como um todo (e.g., funções e parâmetros de pré-processamento).

Para ter uma noção dos experimentos feitos e dos resultados, faz-se a exibição do melhor resultado de cada execução de experimento.

In [3]:
columns_to_show = ['experiment_name', 'tags.mlflow.runName', 'tags.mlflow.parentRunId', 'run_id', 'experiment_id', 'params.model_name',
                   'metrics.f1', 'metrics.precision', 'metrics.recall', 'metrics.training_time']

(best_results
 .assign(experiment_name=lambda f: f['experiment_id'].apply(lambda id: mlflow_client.get_experiment(id).name))
 .sort_values(by='metrics.f1', ascending=False) 
 .loc[lambda f: ~f['params.model_name'].isna()] 
 .drop_duplicates(['tags.mlflow.parentRunId'])
 [columns_to_show]
)

,experiment_name,tags.mlflow.runName,tags.mlflow.parentRunId,run_id,experiment_id,params.model_name,metrics.f1,metrics.precision,metrics.recall,metrics.training_time
45,01_SupervisedClassification,"01_0_Concat Title and Tags + Weight, Price, and Minimum Quantity_SVC-RBF",7d31ba2aa87f4043b216600ab44d2a3a,8c41271d85544a20b02772604ecc5262,0,SVC-RBF,0.817958,0.862407,0.784987,175.712107
87,01_SupervisedClassification,"01_1_Complete Set: Title, Concatenated Tags, Weight, Price, and Minimum Quantity_SVC-RBF",8a594d3cc32440edadabfe9cd5c57833,4517a510ae0b4b9796629c43c94f77ad,0,SVC-RBF,0.816587,0.863858,0.781765,435.923445
16,01_SupervisedClassification,01_0_Concat Title and Tags + Price_LGB,ec06415edba04014a170e44f491d6cd3,ad4ad8aa0405418fb551d15803e41f09,0,LGB,0.806909,0.844251,0.778679,13.337812
70,01_SupervisedClassification,"01_0_Intermediate Set: Title, Weight, Price, and Minimum Quantity_SVC-RBF",fe08d09b23cf48e0aa7d8a28ee9304ff,ad1120ec1d7e4c8eb2c75b1d472d0ddb,0,SVC-RBF,0.795154,0.847650,0.761815,199.268800
104,01_SupervisedClassification,"01_2_Basic Set: Title, Weight_SVC-RBF",f1b9a0390129475c91054cf4dba82146,af88fe8e130c41fc9b78404f3b315ed0,0,SVC-RBF,0.795154,0.847650,0.761815,210.422097


Como mostram os resultados, foram feitos experimentos combinando diferentes atributos disponíveis. Para a maior parte deles, **o algoritmo SVC-RBF (uma implementação do SVM com kernel RBF) teve a maior parte dos melhores resultados**. **Apesar da boa eficácia do algoritmo, pode-se notar que sua eficiência é pelo menos 12 vezes pior do que a do LightGBM (LGB)**. Apesar de essa comparação não ser perfeitamente justa, considerando o modelo do LGB tem 301 *features* em vez de 303, o experimento com título `01_2_Basic Set: Title, Weight_SVC-RBF` apresenta o mesmo número de *features* e a eficácia do SVC-RBF é ainda pior.

Um ponto interessante de ser notado é que concatenar *título* e *tags* tornou o modelo melhor do que a versão com um *embedding* para cada uma das colunas. Isso é interessante pelo fato de melhorar o resultado e ainda reduzir a utilização de recursos na representação dos dados e no treinamento do modelo.

Considerando a grande diferença de eficiência entre o LGB e o SVC-RBF, faz-se a **escolha do LGB como algoritmo principal**. Apesar de ter uma perda de eficácia, é preciso levar em consideração que no cenário real o volume de dados deveria ser consideravelmente maior e que a implementação do SVM não linear impõe restrições. Assim, a seguir, são recuperados os resultados do LGB em todos os experimentos.

In [4]:
(best_results
 .assign(experiment_name=lambda f: f['experiment_id'].apply(lambda id: mlflow_client.get_experiment(id).name))
 .sort_values(by='metrics.f1', ascending=False) 
 .loc[lambda f: f['params.model_name'] == 'LGB' ]
 .drop_duplicates(['tags.mlflow.parentRunId'])
 [columns_to_show]
)

,experiment_name,tags.mlflow.runName,tags.mlflow.parentRunId,run_id,experiment_id,params.model_name,metrics.f1,metrics.precision,metrics.recall,metrics.training_time
41,01_SupervisedClassification,"01_0_Concat Title and Tags + Weight, Price, and Minimum Quantity_LGB",7d31ba2aa87f4043b216600ab44d2a3a,2583b7bdff534ce2a8559c610c447306,0,LGB,0.808534,0.847163,0.779735,16.645187
16,01_SupervisedClassification,01_0_Concat Title and Tags + Price_LGB,ec06415edba04014a170e44f491d6cd3,ad4ad8aa0405418fb551d15803e41f09,0,LGB,0.806909,0.844251,0.778679,13.337812
83,01_SupervisedClassification,"01_1_Complete Set: Title, Concatenated Tags, Weight, Price, and Minimum Quantity_LGB",8a594d3cc32440edadabfe9cd5c57833,e035e8b3fa174b7baf1d08df4080eb5e,0,LGB,0.805640,0.853343,0.771814,26.536993
116,01_SupervisedClassification,"01_0_Basic Set: Title, Weight_LGB",f1b9a0390129475c91054cf4dba82146,79b0e25a5b52439aabf7e7e08931c1ab,0,LGB,0.772763,0.833968,0.735516,14.136489
50,01_SupervisedClassification,"01_2_Intermediate Set: Title, Weight, Price, and Minimum Quantity_LGB",fe08d09b23cf48e0aa7d8a28ee9304ff,b953bd5a2dad43bc907eb9d8ca5d12eb,0,LGB,0.772763,0.833968,0.735516,12.568504


Como é possível observar, o modelo com melhor resultado teve a concatenação de título e tags, representada por *embeddings*, e três atributos numéricos: *peso*, *preço* e *quantidade mínima*. Como cada *feature* adicional no modelo representa algo a mais a ser mantido e a exigir um trabalho de garantia de qualidade, o modelo mais simples, apenas com preço, é escolhido. Isso também ajuda a minimizar o impacto da ausência dos dados de *peso* e *quantidade minima* na utilização do modelo para a classificação da categoria das buscas.

## Restauração de Experimentos

A partir da escolha de uma execução individual, é possível restaurar os elementos utilizados na experimentação para aplicá-los aos dados.

In [5]:
# Restaura o experimento pelo maior valor de F1 Médio (entre as classes)
best_experiments_results = mlflow.search_runs(experiment_ids=[EXPERIMENT_ID], max_results=100, order_by=['metrics.f1 DESC'], filter_string='attributes.status="FINISHED"')
best_experiment = (best_experiments_results
 .loc[lambda f: f['run_id'] == 'ad4ad8aa0405418fb551d15803e41f09']
 .iloc[0]
)

# Recupera o ID do artefato para recuperar modelos e recursos persistidos para o experimento
artifact_uri = best_experiment["artifact_uri"]

# Mostra as principais informações da execução
display(HTML('<h4>Resultado Escolhido:</h4>'))
for name, key in [('Run ID', 'run_id'),
                  ('Model Name', 'params.model_name'),
                  ('Average F1', 'metrics.f1'),
                  ('Average Precision', 'metrics.precision'),
                  ('Average Recall', 'metrics.recall'),
                  ('Experiment Run', 'tags.mlflow.runName'),
                 ]:
    display(HTML(f'<li><strong>{name}</strong>: {best_experiment[key]}</li>'))

# Recupera o pacote de funções e parâmetros de pré-processamento dos dados


preprocessing_model_path = str(Path(best_experiment['artifact_uri']
                                    .replace(best_experiment['run_id'],
                                             best_experiment['tags.mlflow.parentRunId'])).joinpath('log',
                                                                                                   'preprocessing_model'))
preprocessing_model = mlflow.pyfunc.load_model(preprocessing_model_path)

# Recupera o modelo treinado
model = mlflow.sklearn.load_model(f'{artifact_uri}/model')

# Recupera o Label Encoder, caso seja preciso avaliar o modelo
label_encoder_path = str(Path(best_experiment['artifact_uri']
                              .replace(best_experiment['run_id'],
                                       best_experiment['tags.mlflow.parentRunId'])).joinpath('label_encoder'))
label_encoder_model = mlflow.sklearn.load_model(label_encoder_path)

Para validar o funcionamento da restauração do modelo, parte dos dados de treinamento são recuperados para uma avaliação.

In [6]:
columns_to_read = ['product_id', 'title', 'concatenated_tags', 'price', 'weight', 'express_delivery', 'minimum_quantity', 'category', 'creation_date']
frame = (pd
         .read_csv(str(Path(settings.DATA_PATH).joinpath('interim', 'training.csv')), usecols=columns_to_read)
         .drop_duplicates()  # Manter mais de uma ocorrência de produto apenas se existir variação nos dados
        )
print(f'Registros carregados: {len(frame)}')

Registros carregados: 32729


Tendo os dados, é possível reprocessar *labels*, *features* e fazer a inferência.

In [7]:
sample_frame = (frame
                .drop_duplicates('category')
                .sort_values(by='category')
               )

# Processa os dados para inferência
features = preprocessing_model.predict(sample_frame)

# Realiza a inferência
sample_frame['pred'] = model.predict(features)

# Codificar labels
sample_frame['label'] = label_encoder_model.transform(sample_frame.category)

display_side_by_side([sample_frame, 
                      pd.DataFrame(features).describe().T.head(10)], 
                     ['Dados Recuperados e Predição', 
                      f'Features (10 de {features.shape[1]})'])

del sample_frame, features

## Análise dos Resultados

Nesta seção faz-se a análise mais aprofundada dos resultados do modelo escolhido. O primeiro passo é restaurar os dados utilizados para criar e validadr os modelos preliminares criados no *notebook* [Classificação de Categorias](03.0_Classificacao_de_Categorias.ipynb). O pré-processamento dos dados pode ser feito antes mesmo da divisão porque os parâmetros do pré-processamento foram feitos com os dados de treinamento, conforme o *notebook* indicado.

In [8]:
ENCODING_SORTED_LABELS = [label_encoder_model.inverse_transform([item])[0]
                          for item in range(0, 6)]

# Pré-processa e faz predições
features = preprocessing_model.predict(frame)

frame['pred'] = model.predict(features)
frame['label'] = label_encoder_model.transform(frame.category)

# Cria coluna com as probabilidades de cada item pertencer a uma categoria
class_prob_frame = pd.DataFrame(model.predict_proba(features), 
                                columns=[f'Prob_{item.replace(" ", "_")}' 
                                         for item in ENCODING_SORTED_LABELS])

# Atribui as colunas ao data frame (concatenar causa mudança de tipos)
for column in class_prob_frame.columns:
    frame[column] = class_prob_frame[column].apply(lambda x: f'{x * 100:.2f}%').to_numpy()

# Separa os conjuntos de treinamento e validação
cut_off_period = '2018-05'
split_frame = set_dataset_split(frame, cut_off_period)

training_frame = split_frame.loc[lambda f: f['group'] != 'test'].drop(columns=['group'])
validation_frame = split_frame.loc[lambda f: f['group'] == 'test'].drop(columns=['group'])

del class_prob_frame

A seguir, as predições de treinamento e validação são usadas para avaliar o comportamento do modelo selecionado, tanto nos dados completos, quanto por cada categoria.

In [9]:
# Calcular métricas para treinamento e validação
metrics = [
    {**{'dataset': 'training'}, **compute_multiclass_classification_metrics(training_frame['label'], training_frame['pred'])},
    {**{'dataset': 'validation'}, **compute_multiclass_classification_metrics(validation_frame['label'], validation_frame['pred'])}
]

# Calcular a média das métricas entre as categorias (não proporcional ao número de elementos)
metrics_frame = (pd.DataFrame(metrics)
                 .assign(precision=lambda f: f['precision'].apply(lambda v: np.mean(v, axis=0)))
                 .assign(recall=lambda f: f['recall'].apply(lambda v: np.mean(v, axis=0)))
                 .assign(f1=lambda f: f['f1'].apply(lambda v: np.mean(v, axis=0)))
                )

individual_metrics_frame = (pd.DataFrame([metrics[1]])
                            [['precision', 'recall', 'f1']]
                            .T)

individual_metrics_frame = (pd
                            .DataFrame(np.stack(individual_metrics_frame[0], axis=0), 
                                       columns=ENCODING_SORTED_LABELS)
                            .T
                            .rename(columns={ix: name
                                            for ix, name in enumerate(['precision', 'recall', 'f1'])})
                            .reset_index()
                            .rename(columns={'index': 'category'})
                            .sort_values(by='category')
                           )

# Calcular a distribuição dos registros por classe
distribution_frame = (training_frame
                      [['category']]
                      .assign(registros=1)
                      .groupby('category')
                      .sum()
                      .reset_index()
                      .sort_values(by='category')
                      .assign(percentual=lambda f: (100 * f['registros'] / f['registros'].sum()).apply(lambda v: f'{v:.2f}%'))
                     )


# Exibir resultados
display_side_by_side([metrics_frame, individual_metrics_frame, distribution_frame],
                     ['Métricas Gerais de Treinamento e Validação', 'Métricas de Validação por Classes', 'Distribuição de Registros por Classes no Treinamento'],
                     padding=50
                    )

del metrics, metrics_frame, individual_metrics_frame, distribution_frame

Como é possível notar, o modelo praticamente acertou todos os casos do conjunto de treinamento, com .99 de F1, mas caiu para .80 nos dados de teste. Isso levanta a possibilidade que de alguma forma de regularização possa ser aplicada para simplificar o modelo e permitir uma generalização maior do que foi aprendido. 

Com relação às classes, há uma perda maior de eficácia na classificação de duas delas: `Outros`  e `Papel e Cia`. Para ajudar a entender melhor o que ocorre, pode-se analisar a matriz de confusão.

In [10]:
display_side_by_side([pd
                      .DataFrame(confusion_matrix(validation_frame['label'], validation_frame['pred']), columns=ENCODING_SORTED_LABELS)
                      .rename(index={i: ENCODING_SORTED_LABELS[i] for i in range(0, 6)})
                     ],
                     ['Matriz de Confusão']
                    )

,Bebê,Bijuterias e Jóias,Decoração,Lembrancinhas,Outros,Papel e Cia
Bebê,877,0,37,100,4,16
Bijuterias e Jóias,1,89,4,10,0,0
Decoração,31,1,985,76,2,8
Lembrancinhas,46,4,59,2657,5,70
Outros,7,3,27,32,96,4
Papel e Cia,6,0,9,122,12,199


Considerando que as linhas são os valores dos rótulos (verdadeiros) e as colunas indicam as predições, é possível notar que `Outros` e `Papel e Cia` são incorretamente classificadas mais frequentemente com  `Lembrancinhas`, que é uma categoria com certa ambiguidade e que possui a maior quantidade de registros no conjunto de dados, responsável por 45% do treinamento. Essa predominância da classe, no entanto, é contornada com o balanceamento dos pesos no momento do treinamento. 

A seguir, são coletados alguns casos de classificação incorreta para entender o que está sendo confundido.

In [11]:
def get_classification_examples(base_frame: pd.DataFrame, label: str, prediction: str, size=15) -> pd.DataFrame:
    encoded_label = label_encoder_model.transform([label])[0]
    encoded_prediction = label_encoder_model.transform([prediction])[0]

    return (base_frame
            .loc[lambda f: (f['label'] == encoded_label) & (f['pred'] == encoded_prediction)]
            .head(size)
           )

display_side_by_side([get_classification_examples(validation_frame, 'Papel e Cia', 'Lembrancinhas')], 
                     ['Exemplos de produtos de <strong>Papel e Cia</strong> incorretamente classificados como <strong>Lembrancinhas</strong>'])

,product_id,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,category,pred,label,Prob_Bebê,Prob_Bijuterias_e_Jóias,Prob_Decoração,Prob_Lembrancinhas,Prob_Outros,Prob_Papel_e_Cia,period
109,8336279,Topper - Elefantinho chevron chá de revelação,topper cha revelacao,2018-06-18 18:19:28,10.670000,3.0,1,40,Papel e Cia,3,5,1.63%,0.00%,0.36%,88.94%,0.00%,9.07%,2018-06
446,9846073,caixinha para lembrancinha - sacolinha aniversário 80 anos,aniver vo 90 anos 90 anos festa vo caixas sacolinha bolsinhas,2018-06-11 09:31:12,11.540000,0.0,1,15,Papel e Cia,3,5,0.00%,0.00%,0.00%,99.99%,0.00%,0.01%,2018-06
578,9072281,Caixa para cerveja dia dos Pais,dia pais,2018-07-12 22:25:40,20.260000,7.0,1,5,Papel e Cia,3,5,0.01%,0.00%,0.03%,92.11%,0.00%,7.85%,2018-07
666,11511720,Senha individual - Convite individual,casamento formatura aniversario casamento noivado,2018-08-21 14:39:14,10.500000,8.0,1,34,Papel e Cia,3,5,0.00%,0.00%,0.02%,99.64%,0.00%,0.34%,2018-08
1076,9432165,PORTA BOMBOM CAMISA - DIA DOS PAIS,dia pais,2018-06-23 22:49:23,9.780000,0.0,1,20,Papel e Cia,3,5,0.03%,0.00%,0.06%,99.81%,0.01%,0.09%,2018-06
1115,12724727,Topo de bolo Batizado,batizado 1a comunhao topo bolo,2018-07-11 09:07:48,28.180000,0.0,1,1,Papel e Cia,3,5,10.30%,0.00%,1.20%,86.04%,0.00%,2.46%,2018-07
1799,13564556,Caixa 4 bombons c/tampa,caixinha casamento caixinha bombom lembrancinha convidados q n vao,2018-06-16 16:53:24,10.710000,20.0,1,50,Papel e Cia,3,5,0.00%,0.00%,0.00%,99.06%,0.00%,0.94%,2018-06
1837,9364413,Caixa para 6 doces Melhores Amigos C/10 un,caixa 6 brigadeiros,2018-08-15 17:46:45,25.350000,188.0,1,1,Papel e Cia,3,5,0.04%,0.00%,2.00%,94.13%,0.00%,3.84%,2018-08
2249,14327291,Aplique LOL,l.o.l lol manu 5 anos,2018-07-14 14:21:28,11.100000,0.0,1,20,Papel e Cia,3,5,0.35%,0.00%,2.78%,96.82%,0.01%,0.03%,2018-07
2396,9072281,Caixa para cerveja dia dos Pais,dia pais,2018-07-12 22:25:40,21.090000,6.0,1,12,Papel e Cia,3,5,0.01%,0.00%,0.03%,92.11%,0.00%,7.85%,2018-07


A seguir, faz-se a análise dos resultados considerando a probabilidade dada pelo modelo para cada classe. Ainda que se saiba que as probabilidades devessem ser calibradas para refletir melhor esse conceito, elas serão levadas em consideração para entender como o modelo determina os resultados.

A julgar pelos exemplos exibidos, o título dos produtos em sua grande maioria podem ser considerados ambíguos ou ambivalência, podendo ser encaixados em diferentes categorias. Essa características, que pode chegar a ser vista como polivalência, pode ser notada na forma como o modelo escolhido define as probabilidades, como no caso de `Topper - Elefantinho chevron chá de revelação`. Esse produto tem o rótulo "Papel e Cia", que recebeu probabilidade de 9%, teve predição como "Lembrancinhas", com 88.9% de probabilidade. A categoria mais provável, por um olhar leigo e superficial, não parece ser um erro claro e poderia ser aceita como uma classificação correta.

Para o produto `Caixa Cartonada`, parece existir uma situação mais clara de produto incorretamente classificado -- apesar de poder se considerar que o produto pode fazer parte de uma lembrancinha. O modelo dá 98% de probabilidade de pertencer à categoria "Lembrancinha" e apenas 1.68% de pertencer a "Papel e Cia".

Pelos exemplos avaliados, ainda que se possa trabalhar na utilização de mais *features*, fazer a aumentação de dados ou tentar ajustar o *threshold* das classes, parece existir um problema maior que é inerente aos dados: a ambiguidade ou polivalência dos produtos. Nesse sentido, poderia fazer mais sentido avaliar a qualidade com relação a *soft-labels* em vez de um único *hard-label*. 

In [12]:
display_side_by_side([get_classification_examples(validation_frame, 'Outros', 'Lembrancinhas')], 
                     ['Exemplos de produtos de <strong>Outros</strong> incorretamente classificados como <strong>Lembrancinha</strong>'])

,product_id,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,category,pred,label,Prob_Bebê,Prob_Bijuterias_e_Jóias,Prob_Decoração,Prob_Lembrancinhas,Prob_Outros,Prob_Papel_e_Cia,period
390,8245994,Kit 10 unidades barbante euroroma,meadas anchor,2018-06-07 10:21:43,149.540000,6006.0,0,6,Outros,3,4,4.46%,0.00%,23.01%,57.94%,13.50%,1.10%,2018-06
2394,8753842,Kit Camiseta Chá Bar,cha cha bar,2018-06-05 14:37:26,67.940000,6.0,1,1,Outros,3,4,0.15%,0.00%,0.17%,99.67%,0.00%,0.01%,2018-06
2440,6006788,Mosquetão Niquelado - 30mm - 10 unidades,ferragens acessorios,2018-07-22 23:08:46,34.079998,5.0,0,0,Outros,3,4,0.01%,1.60%,3.47%,64.88%,29.84%,0.20%,2018-07
3093,11356309,Lembrancinha Dia dos Pais,dia pais,2018-06-03 16:20:45,15.560000,5.0,1,1,Outros,3,4,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,2018-06
3272,865802,sabonete camomila,sabonetes lembrancinhas,2018-08-02 16:24:30,14.110000,6.0,1,5,Outros,3,4,0.01%,0.00%,0.01%,99.98%,0.00%,0.00%,2018-08
3344,6414970,Aplique Bigode,dia pais,2018-07-09 10:07:05,9.490000,9.0,1,100,Outros,3,4,3.08%,0.00%,7.79%,87.86%,0.63%,0.63%,2018-07
3696,7170633,Lembrancinha Dia dos Pais,dia pais,2018-06-04 15:28:53,15.380000,5.0,1,5,Outros,3,4,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,2018-06
3791,11362208,Kit viagem com tag personalizada,kits niver,2018-08-03 17:39:52,19.610000,6.0,1,6,Outros,3,4,0.06%,0.00%,0.01%,50.43%,0.00%,49.49%,2018-08
4457,3711214,FRETE GRATIS Kit Atividade Educacional Montessori brinquedo,hastes coloridas atividades ludicas pompons acrilicos escola,2018-08-28 12:41:44,75.430000,0.0,0,10,Outros,3,4,3.73%,0.00%,3.65%,88.69%,0.02%,3.91%,2018-08
5632,3645138,SABONETE FLOR DE CEREJEIRA,kokeshi,2018-06-21 14:37:23,40.980000,135.0,1,10,Outros,3,4,0.07%,0.00%,12.02%,87.79%,0.12%,0.00%,2018-06


Para a categoria "Outros", que deve enquadrar produtos variados que não se encaixam nas demais categorias, o modelo apresentou menos certeza, considerando o valor máximo das probabilidades. O caso com maior dúvida é `Kit viagem com tag personalizada`, que possui 50.4% de probabilidade para a categoria "Lembrancinhas" e 49.49% de pertencer à categoria "Papel e Cia". Novamente, pode-se ver alguma relação com as categorias incorretamente classificadas. Considerando  que "Outros" pode ter muita diversidade de conteúdo, é de se entender que o modelo veja mais semelhança entre produtos de outras categorias que estão relacionados a festividades e presentes.